In [75]:
import asyncio
import pandas as pd
from selection_service.enums.Enums import DesignCode, ProviderName
from selection_service.core.Pipeline import EarthquakeAPI, EarthquakePipeline, PipelineContext
from selection_service.providers.Providers import IDataProvider, ProviderFactory
from selection_service.processing.Selection import SelectionConfig,SearchCriteria,TBDYSelectionStrategy,TargetParameters
import logging
from selection_service.core.LoggingConfig import setup_logging

setup_logging()  # INFO default

In [76]:
providerFactory = ProviderFactory()
afadProvider = providerFactory.create_provider(
    ProviderName.AFAD, 
    station_file_path="D:\\github\\SelectionEarthquake\\data\\stations.xlsx"
)
peerProvider = providerFactory.create_provider(
    provider_type=ProviderName.PEER, 
    file_path="D:\\github\\SelectionEarthquake\\data\\NGA-West2_flatfile.csv"
)
con = SelectionConfig(design_code=DesignCode.TBDY_2018, num_records=22, max_per_station=3, max_per_event=3, min_score=55)
strategy = TBDYSelectionStrategy(config=con)
search_criteria = SearchCriteria(start_date="2000-01-01", end_date="2025-09-05", min_magnitude=7.0, max_magnitude=10.0, min_vs30=400, max_vs30=500 )
target_params = TargetParameters(magnitude=7.0, distance=30.0, vs30=400.0, pga=300, mechanism=["StrikeSlip","Reverse"] )

In [77]:
api = EarthquakeAPI(providers=[afadProvider,peerProvider], strategies=[strategy])

In [78]:
result = api.run_sync(criteria=search_criteria, target=target_params, strategy_name=strategy.get_name())

AFAD arama kriterleri: {'startDate': '2000-01-01T00:00:00.000Z', 'endDate': '2025-09-05T23:59:59.999Z', 'fromMagnitude': 7.0, 'toMagnitude': 10.0, 'fromVs30': 400, 'toVs30': 500}
AFAD'dan 117 kayıt alındı.
PEER'dan 279 kayıt alındı.


--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\hp-hp\AppData\Local\Programs\Python\Python313\Lib\logging\__init__.py", line 1154, in emit
    stream.write(msg + self.terminator)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp-hp\AppData\Local\Programs\Python\Python313\Lib\encodings\cp1254.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f3c6' in position 26433: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\github\SelectionEarthquake\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\github\SelectionEarthquake\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.s

In [79]:
type(result)

selection_service.processing.ResultHandle.Result

In [80]:
type(result.value)

selection_service.core.Pipeline.PipelineResult

In [81]:
result.success

True

In [82]:
result.value.failed_providers

[]

In [83]:
result.value.execution_time

3.3726885318756104

In [84]:
result.value.logs

['[OK] AFAD success',
 '[OK] PEER success',
 'Combined 2 datasets, total 396 records',
 '🏆 Strategy applied: TBDY_2018',
 '⏱ Execution time: 3.37 sec']

In [85]:
target_params

TargetParameters(magnitude=7.0, distance=30.0, vs30=400.0, mechanism=['StrikeSlip', 'Reverse'], pga=300, pgv=None, t90=None)

In [86]:
result.value.selected_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','MAGNITUDE_TYPE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,MAGNITUDE_TYPE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
118,PEER,139,"Tabas, Iran",1978,7.35,Mw,Dayhook,471.53,13.940000,Reverse,0.000000,326.924291,25.872000,100.000000
112,AFAD,327918,17966,2023,7.70,MW,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348
60,AFAD,327952,17966,2023,7.70,MW,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
61,AFAD,327966,17966,2023,7.70,MW,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348
369,PEER,8166,"Duzce, Turkey",1999,7.14,Mw,IRIGM 498,425.00,3.580000,StrikeSlip,0.000000,365.542879,25.269000,87.826087
262,PEER,4451,"Montenegro, Yugo.",1979,7.10,Mw,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
263,PEER,4452,"Montenegro, Yugo.",1979,7.10,Mw,Debar - Skupstina Opstine,485.04,118.490000,Reverse,0.000000,55.347752,2.480000,69.565217
259,PEER,3821,Hector Mine,1999,7.13,Mw,San Pedro - Palos Verdes,477.92,206.310000,StrikeSlip,0.000000,8.361444,4.209900,69.565217
258,PEER,3798,Hector Mine,1999,7.13,Mw,Los Angeles - Colorado & Eagle Rock,418.83,177.390000,StrikeSlip,0.000000,20.845015,3.979400,69.565217
395,PEER,8605,El Mayor-Cucapah,2010,7.20,Mw,Bird Spring,402.00,388.790000,StrikeSlip,0.000000,2.152265,2.253900,69.565217


In [87]:
result.value.scored_df[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
0,AFAD,328381,17969,2023,7.6,İnönü Ortaokulu,444.0,88.355651,StrikeSlip,35.716667,42.899350,11.014204,60.869565
1,AFAD,328392,17969,2023,7.6,Meteoroloji İstasyon Müdürlüğü,420.0,108.924054,StrikeSlip,18.980000,52.261725,7.239375,60.869565
2,AFAD,328348,17969,2023,7.6,TÜRK TELEKOM Erkek Öğrenci Pansiyonu,445.0,8.647423,StrikeSlip,35.376667,93.367520,16.699829,60.869565
3,AFAD,328353,17969,2023,7.6,"ÇUKOBİRLİK, Gaziantep Yolu",485.0,11.361235,StrikeSlip,41.053333,73.751004,28.684273,60.869565
4,AFAD,328366,17969,2023,7.6,Çok Programlı Lise,430.0,33.239312,StrikeSlip,38.856667,65.873710,16.061071,60.869565
...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,PEER,8585,El Mayor-Cucapah,2010,7.2,Santa Barbara,485.0,417.450000,StrikeSlip,0.000000,1.808346,1.543500,69.565217
392,PEER,8587,El Mayor-Cucapah,2010,7.2,"San Clemente Island 2, Ca, Usa",442.0,256.830000,StrikeSlip,0.000000,3.907067,1.871500,69.565217
393,PEER,8592,El Mayor-Cucapah,2010,7.2,Slate Mountain,482.0,386.450000,StrikeSlip,0.000000,2.546297,1.134900,69.565217
394,PEER,8603,El Mayor-Cucapah,2010,7.2,Vincent Substation,405.0,298.860000,StrikeSlip,0.000000,4.136837,2.015600,69.565217


In [88]:
result.value.scored_df.sort_values(by=['SCORE'],ascending=True)[['PROVIDER','RSN','EVENT','YEAR','MAGNITUDE','STATION','VS30(m/s)','RRUP(km)','MECHANISM','T90_avg(sec)','PGA(cm2/sec)','PGV(cm/sec)','SCORE']]

,PROVIDER,RSN,EVENT,YEAR,MAGNITUDE,STATION,VS30(m/s),RRUP(km),MECHANISM,T90_avg(sec),PGA(cm2/sec),PGV(cm/sec),SCORE
266,PEER,4636,"Wenchuan, China",2008,7.90,Longxian,429.21,269.980000,Reverse/Oblique,0.000000,113.070674,19.055000,26.086957
286,PEER,4745,"Wenchuan, China",2008,7.90,Shimiancaoke,416.07,198.600000,Reverse/Oblique,0.000000,36.973032,3.364400,26.086957
284,PEER,4739,"Wenchuan, China",2008,7.90,Maerkangdizhenju,409.53,120.250000,Reverse/Oblique,0.000000,48.558608,2.861200,26.086957
283,PEER,4738,"Wenchuan, China",2008,7.90,Maerkangdiban,474.67,129.380000,Reverse/Oblique,0.000000,26.627997,2.528700,26.086957
282,PEER,4733,"Wenchuan, China",2008,7.90,Ludingshuichang,414.03,144.770000,Reverse/Oblique,0.000000,47.358274,2.959600,26.086957
...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,PEER,4451,"Montenegro, Yugo.",1979,7.10,Bar-Skupstina Opstine,462.23,6.980000,Reverse,0.000000,361.728092,49.588000,87.826087
60,AFAD,327952,17966,2023,7.70,Çok Programlı Anadolu Lisesi,448.00,20.737167,StrikeSlip,7.693333,291.290941,98.907393,91.304348
61,AFAD,327966,17966,2023,7.70,Ali Tekden Sağlık Ocağı,424.00,38.529217,StrikeSlip,31.463333,286.722564,48.332219,91.304348
112,AFAD,327918,17966,2023,7.70,"Necip Fazıl Kültür Merkezi, Alparslan Türkeş B...",484.00,37.225843,StrikeSlip,43.853333,320.930444,37.353823,91.304348


In [89]:
result.value.scored_df.describe().T

,count,mean,std,min,25%,50%,75%,max
RSN,396.0,97263.419192,147093.885767,13.000000,1577.500000,5844.500000,327992.250000,328703.000000
YEAR,396.0,2008.674242,11.094642,1952.000000,1999.000000,2010.000000,2023.000000,2023.000000
MAGNITUDE,396.0,7.493990,0.276636,7.000000,7.200000,7.600000,7.700000,7.900000
STATION_LAT,396.0,29.827815,18.133514,-46.250700,25.037650,34.147735,37.386760,65.311100
STATION_LON,396.0,23.262131,99.502983,-149.951200,-116.756000,36.485742,120.551850,174.915800
VS30(m/s),396.0,433.747500,81.721624,0.000000,420.000000,444.000000,470.005000,500.000000
STRIKE1,396.0,209.429293,118.997024,20.000000,85.100000,233.000000,307.500000,358.000000
DIP1,396.0,60.520202,19.326878,14.000000,50.000000,63.000000,74.000000,90.000000
RAKE1,396.0,59.037879,119.228134,-178.000000,18.000000,55.000000,174.000000,180.000000
RJB(km),396.0,185.195076,151.573197,0.000000,59.317500,147.581765,274.485855,713.780000
